In [3]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.cluster import KMeans

In [5]:
products = pd.read_csv('../data/products.csv')
orders = pd.read_csv('../data/orders.csv')
departments = pd.read_csv('../data/departments.csv')
aisles = pd.read_csv('../data/aisles.csv')
train = pd.read_csv('../data/order_products__train.csv')
prior = pd.read_csv('../data/order_products__prior.csv')

### Concat Prior and Train

In [8]:
priorTrain = pd.concat([prior, train])

In [9]:
priorTrain.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### Merge priorTrain with Orders

In [12]:
df = pd.merge(priorTrain, orders, how="left", left_on="order_id", right_on="order_id").drop(['eval_set'], axis=1)

In [14]:
df.sample(10)

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
21694138,2288331,26604,1,1,138693,6,0,13,30.0
26628737,2807961,47766,10,0,54671,6,0,11,6.0
1292377,136334,39561,6,1,179727,6,1,13,8.0
20751367,2189025,16659,3,1,137068,8,1,13,30.0
31331952,3304697,17616,1,1,110746,11,1,13,9.0
17843197,1881995,30995,2,1,53198,3,5,9,7.0
30731029,3241504,5337,4,1,123160,41,6,19,7.0
18881502,1991508,27323,18,0,159869,4,4,12,25.0
33508357,2654486,44142,4,0,202431,4,5,11,30.0
28328044,2987500,12320,15,0,21240,3,5,7,8.0


In [21]:
df.shape

(33819106, 9)

# EDA

### Summary - Products per Order by User

In [103]:
# Total numbers per order
products_per_order = df.groupby(['user_id','order_id'], as_index=False)[['add_to_cart_order']].max()
# Products per order
products_per_order = products_per_order.groupby(['user_id'], as_index=False)[['add_to_cart_order']].agg(['count', 'sum', 'min', 'max', 'median', 'mean', 'std'])
# Rename columns
products_per_order.columns = ['total_orders','total_products','min_products','max_products','med_products','avg_products','std_products']
products_per_order

,total_orders,total_products,min_products,max_products,med_products,avg_products,std_products
user_id,,,,,,,
1,11,70,4,11,6.0,6.363636,2.110579
2,15,226,5,31,14.0,15.066667,7.055562
3,12,88,5,11,7.0,7.333333,2.103388
4,5,18,2,7,3.0,3.600000,2.073644
5,5,46,5,12,9.0,9.200000,2.683282
...,...,...,...,...,...,...,...
206205,4,51,7,19,12.5,12.750000,6.130525
206206,67,285,1,24,4.0,4.253731,3.230270
206207,16,223,5,25,13.0,13.937500,6.038419


### Summary - Reordered Products per Orders by Users

In [104]:
# Total items per order
items_per_order = df.groupby(['user_id','order_id'], as_index=False)[['add_to_cart_order']].max()
# Total items reordered per order
reorders_per_order = df.groupby(['user_id','order_id'], as_index=False)[['reordered']].sum()
# Combine items/order and reoredered items/order
reorders = pd.merge(items_per_order, reorders_per_order, how="left")
# Percent of items are reordered
reorders['percent_reordered'] = reorders['reordered']/reorders['add_to_cart_order']
# Summary stats
reorders = reorders.groupby(['user_id'], as_index=False)[['percent_reordered']].agg(['max', 'median', 'mean', 'std'])
# Rename columns
reorders.columns=['max_reordered','med_reordered','avg_reordered','std_reordered']
reorders

,max_reordered,med_reordered,avg_reordered,std_reordered
user_id,,,,
1,1.000000,0.666667,0.724311,0.307419
2,0.888889,0.562500,0.443903,0.304708
3,1.000000,0.763889,0.658817,0.333725
4,0.142857,0.000000,0.028571,0.063888
5,0.666667,0.444444,0.391111,0.242212
...,...,...,...,...
206205,0.857143,0.282895,0.355733,0.360894
206206,1.000000,0.500000,0.570676,0.312977
206207,1.000000,0.645833,0.637144,0.310794


### Summary - Days Between Orders by User

In [108]:
# Days between orders
days = df.groupby(['user_id'], as_index=False)[['days_since_prior_order']].agg(['count', 'min', 'max', 'median', 'mean', 'std'])
# Rename columns
days.columns=['total_days','min_days','max_days','med_days','avg_days','std_days']
days

,total_days,min_days,max_days,med_days,avg_days,std_days
user_id,,,,,,
1,65,0.0,30.0,19.0,19.200000,8.791331
2,213,3.0,30.0,13.0,18.009390,9.778196
3,78,7.0,21.0,10.0,11.487179,4.869048
4,14,0.0,21.0,20.0,15.357143,8.580901
5,35,6.0,19.0,11.0,12.314286,5.251170
...,...,...,...,...,...,...
206205,34,10.0,30.0,10.0,14.705882,8.611231
206206,281,0.0,15.0,3.0,4.042705,3.453511
206207,199,1.0,30.0,16.0,14.879397,11.293850


### Summary - Day and Hour by User

In [109]:
# Modes of day of week and hour of day
day_and_hour = df.groupby(['user_id'])[['order_dow','order_hour_of_day']].agg(pd.Series.mode)
# Rename columns
day_and_hour.columns = ['mode_day_of_week','mode_hour_of_day']
day_and_hour

,mode_day_of_week,mode_hour_of_day
user_id,,
1,4,8
2,1,11
3,0,16
4,4,15
5,0,18
...,...,...
206205,1,16
206206,0,18
206207,1,12


### Merge Summaries

In [118]:
user_summary = pd.merge(products_per_order, reorders, how='left', left_index=True, right_index=True)
user_summary = pd.merge(user_summary, day_and_hour, how='left', left_index=True, right_index=True)
user_summary = pd.merge(user_summary, days, how='left', left_index=True, right_index=True)
user_summary

,total_orders,total_products,min_products,max_products,med_products,avg_products,std_products,max_reordered,med_reordered,avg_reordered,std_reordered,mode_day_of_week,mode_hour_of_day,total_days,min_days,max_days,med_days,avg_days,std_days
user_id,,,,,,,,,,,,,,,,,,,
1,11,70,4,11,6.0,6.363636,2.110579,1.000000,0.666667,0.724311,0.307419,4,8,65,0.0,30.0,19.0,19.200000,8.791331
2,15,226,5,31,14.0,15.066667,7.055562,0.888889,0.562500,0.443903,0.304708,1,11,213,3.0,30.0,13.0,18.009390,9.778196
3,12,88,5,11,7.0,7.333333,2.103388,1.000000,0.763889,0.658817,0.333725,0,16,78,7.0,21.0,10.0,11.487179,4.869048
4,5,18,2,7,3.0,3.600000,2.073644,0.142857,0.000000,0.028571,0.063888,4,15,14,0.0,21.0,20.0,15.357143,8.580901
5,5,46,5,12,9.0,9.200000,2.683282,0.666667,0.444444,0.391111,0.242212,0,18,35,6.0,19.0,11.0,12.314286,5.251170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206205,4,51,7,19,12.5,12.750000,6.130525,0.857143,0.282895,0.355733,0.360894,1,16,34,10.0,30.0,10.0,14.705882,8.611231
206206,67,285,1,24,4.0,4.253731,3.230270,1.000000,0.500000,0.570676,0.312977,0,18,281,0.0,15.0,3.0,4.042705,3.453511
206207,16,223,5,25,13.0,13.937500,6.038419,1.000000,0.645833,0.637144,0.310794,1,12,199,1.0,30.0,16.0,14.879397,11.293850


In [119]:
user_summary.to_csv('../data/user_summary.csv') 